Before you turn this problem in, make sure everything runs as expected. First, **restart the kernel** (in the menubar, select Kernel$\rightarrow$Restart) and then **run all cells** (in the menubar, select Cell$\rightarrow$Run All).

Make sure you fill in any place that says `YOUR CODE HERE` or "YOUR ANSWER HERE", as well as any collaborators you worked with:

In [1]:
COLLABORATORS = ""

---

In [2]:
%matplotlib inline
%precision 16
import numpy
import matplotlib.pyplot as plt
import pandas as pd

# Final Project

This notebook will provide a brief structure and rubric for presenting your final project. 

The purpose of the project is 2-fold
* To give you an opportunity to work on a problem you are truly interested in (as this is the best way to actually learn something)
* To demonstrate to me that you understand the overall workflow of problem solving from problem selection to implementation to discussion 

You can choose any subject area that interests you as long as there is a computational component to it.  However, please do not reuse projects or homeworks you have done in other classes.  This should be **your** original work.

**You can work in teams, but clearly identify each persons contribution** and every team member should hand in their own copy of the notebook.

### Structure
There are 5 parts for a total of 100 points that provide the overall structure of a mini research project.

* Problem Description
* Problem Justification
* Description of Computational components needed to address problem
* Implementation including tests
* Discussion of results and future directions

For grading purposes, please try to make this notebook entirely self contained. 

The project is worth about 2 problem sets and should be of comparable length or less (please: I will have about 100 of these to read and I am not expecting full 10 page papers).  The actual project does not necessarily have to work but in that case you should demonstrate that you understand why it did not work and what steps you would take next to fix it.

Have fun

## Problem Description [15 pts]

In 2-4 paragraphs, describe the general problem you want to solve and the goals you hope to achieve. You should provide any relevant background and references, particularly if you are reproducing results from a paper.  Please use proper spelling and grammar. 

The continuity equation depends on the application of devices, we will solve the computational problem for photoelectronic diodes.

Excess carrier can be generated in neutral N and P region of the semiconductor.  The distribution of excess minority carriers within P region can be calculated by **Bipolar Transportation Equation**, Which is:

$$
D_{n} \frac{\partial^{2}p}{\partial x^{2}}+G_{L}-\frac{p}{\tau_{p0}}=\frac{\partial p}{\partial t}
$$

Assume that the electric field in the neutral zone (non-space-charge region) is zero. We only focus the equilibrium state $\partial p/\partial t=0$. Now the continuity equation becomes an ODE:

$$
\frac{\mathrm{d}^{2}p}{\mathrm{d} x^{2}}-\frac{p}{L_{p}^{2}}=-\frac{G_{L}}{D_{p}}
$$
Where$L^2_p=D_p \tau_{p0}$.

The solution of this equation has two part: general solution and particular solution. The general solution can be obtained by following equation:

$$
\frac{d^{2}p_h}{d x^{2}}-\frac{p_h}{L_{p}^{2}}=0
$$
Where $p_h$ is the general solution, calculated by:
$$
p_h=A \mathrm{e}^{-x / L_{p}}+B \mathrm{e}^{+x / L_{p}} \quad(x \geqslant 0)
$$

One boundary condition is that $p_h$ must be finite, which means for semiconductors have long length $B=0$.

The particular solution can be obtained by:
$$
-\frac{p_p}{L_{p}^{2}}=-\frac{G_{L}}{D_{p}}
$$

Therefore we could get:
$$
p_p=\frac{G_{L} L_{p}^{2}}{D_{p}}=\frac{G_{L}\left(D_{p} \tau_{p0}\right)}{D_{p}}=G_{L} \tau_{p0}
$$

## Problem Justification [5 pts] 

Briefly describe why this problem is important to you,  and, if possible, to anyone else.

## Computational  Methods [10 pts]

Briefly describe the specific approach you will take to solve some concrete aspect of the general problem. 

You should  include all the numerical or computational methods you intend to use.  These can include methods or packages  we did not discuss in class but provide some reference to the method. You do not need to explain how the methods work, but you should briefly justify your choices. 

**If you need to install or import any additional python packages,  please provide complete installation instructions in the code block below**



YOUR ANSWER HERE

In [3]:
# Provide complete installation or import information for external packages or modules here e.g.

#pip install somepackage
# from somepackage import blah

## Implementation [60 pts]

Use the Markdown and Code blocks below to implement and document your methods including figures.  Only the first code block will be a grading cell but please add (not copy) cells in this section to organize your work. 

Please make the description of your problem readable by interlacing clear explanatory text with code. 
All code should be well described and commented.

For at least one routine you code below, you should provide a test block (e.g. that implements `numpy.testing` routines) to validate your code.

YOUR ANSWER HERE

In [5]:
# Compute p0 for each material
kT = 0.025852
Eg = [1.12, 0.66, 1.424]
GL = 10. ** 20.

# Product of NaNc
Ni[0] = (1.5 * 10. ** 10) ** 2 
Ni[1] = (1.8 * 10. ** 6) ** 2
Ni[2] = (2.4 * 10. ** 13) ** 2

# Lifespan
tau = [30. * 10. ** (-6.), 140. * 10. ** (-6.), 10. * 10. ** (-9.)]

# Given major carrier concentration
impurity = [10. ** 13, 10. ** 14, 10. ** 15]
Si_p0 = numpy.empty(3)
Ge_p0 = numpy.empty(3)
GaAs_p0 = numpy.empty(3)

# Compute minor carrier concentration
for i in range(3):
    Si_p0[i] = Ni[0] / impurity[i]
    Ge_p0[i] = Ni[1] / impurity[i]
    GaAs_p0[i] = Ni[2] / impurity[i]

#print(Si_p0)

# Diffusion Constant
diff_const = [6.47, 44., 10.]

# Diffusion Length
Lp = numpy.empty(3)
for i in range(3):
    Lp[i] = tau[i] * diff_const[i]

#donor concentration
donor_conc = [10. * 10 ** 13, 10. * 10 ** 14, 10. * 10 ** 15]

x = numpy.linspace(0,1,100)


# Shooting method
def shoot_bvp(f, x, u_a, u_b, rtol=1.e-5, atol=1.e-9):
    """
    Solve the two-point boundary value problem on the interval x\in [a,b], using a shooting method that combines 
        scipy.integrate.solve_ivp and scipy.optimize.root_scalar and allows a range of boundary conditions
        
        
    parameters:
    -----------
    f: calleable 
        vector value function for righthand side of the ODE with interface f(t,u). returns ndarray of length 2
    x: numpy array
        coordinates array for solution  on interval [a,b] with x[0] = a, x[-1] = b
    u_a:  numpy array (length 2)
        provide initial boundary conditions  [u, u' ] at x=a
    u_b:  numpy array (length 2)
        target boundary condition at x = b
    i_a: integer
        index of known boundary condition at x = a.  i.e.
        if dirichlet conditions : i_a = 0 and u(a) is known  
        if neumann conditions   : i_a = 1 and u'(a) is known
        the complementary index is adjusted to match the boundary condition at b
    i_b: integer
        index of known boundary condition at x = b. i.e.
        if dirichlet conditions : i_b = 0 and u(b) is known  
        if neumann conditions   : i_b = 1 and u'(b) is known
        the complementary index is ignored at b
    rtol:  float
        relative tolerance
    atol:  float
        absolute tolerance
        
    returns:
    --------
    u: solution u(x) for x (uses dense_output from solve_ivp to interpolate solution onto x)
    """
    
# YOUR CODE HERE

    # Initial Conditions and guess
    u0_rhs = u_b
    u_lfs = u_a
    
    def convergence(guess):
        u0 = [u_lfs, guess] 
        sol = solve_ivp(f, [x[0], x[-1]], u0, rtol = rtol, atol = atol)
        
        u_end = sol.y[0,-1]
        return numpy.max(numpy.abs(u_end - u0_rhs))
    
    sol = root_scalar(convergence,x0 = -1.0, x1 = 1.0,xtol=atol, rtol=rtol)
    guess = sol.root
    
    u0 = [u_lfs, guess]
            
    sol = solve_ivp(f, [x[0], x[-1]], u0, dense_output=True, rtol = rtol, atol = atol)
    u = sol.sol(x)[0]
    
    return u

# Define f for Silicon
def f0(x, v):
    return numpy.array([v[1], v[0] / Lp[0] ** 2 - GL / diff_const[0]])

# Define f for Germanium
def f1(x, v):
    return numpy.array([v[1], v[0] / Lp[1] ** 2 - GL / diff_const[1]])

# Define f for Gallium Arsenide
def f2(x, v):
    return numpy.array([v[1], v[0] / Lp[2] ** 2 - GL / diff_const[2]])
    
# Define initial conditions for Silicon
u_a = - Si_p0
u_b = 0
sol_Si = numpy.empty([3,len(x)])

# Solution for Silicon
for i in range(3):
    sol_Si[i,:] = shoot_bvp(f0, x, u_a[i], u_b, rtol=1.e-4, atol=1.e-3)
    
fig = plt.figure(figsize=(16,6))
axes = fig.add_subplot(1, 1, 1)
axes.plot(x, sol_Si[0], label="u_sol")
#Exact solution
p = lambda x: tau[0] * GL - (GL * tau[0] + Si_p0[0]) * numpy.exp(- x / numpy.sqrt(diff_const[0] * tau[0]))
#axes.plot(x, p(x), label="u_true")

#axes.set_ylim((-Si_p0[0], 0))
#axes.set_xlim((0., 0.1))
axes.set_xlabel("x")
axes.set_ylabel("$p$")
axes.legend(loc='best')
axes.grid()  

NameError: name 'root_scalar' is not defined

## Discussion [10 pts]

Discuss the results of your code including 
* Why do you believe that your numerical results are correct (convergence, test cases etc)?
* Did the project work (in your opinion)?
* If yes:  what would be the next steps to try
* If no:  Explain why your approach did not work and what you would do differently next time


YOUR ANSWER HERE